In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [ ]:
#dbutils.secrets.listScopes()

In [ ]:
#dbutils.secrets.list(scope="databricksscope")

In [ ]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [ ]:
display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Claims.csv,Claims.csv,5766,1716728851000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/disease.csv,disease.csv,1489,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/group.csv,group.csv,4390,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/hospital.csv,hospital.csv,1528,1716728950000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1716718820000


In [ ]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Claims.csv',header=True)#,inferSchema=True)

In [ ]:
data.show(5,False)

+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
|claim_id|patient_id|disease_name  |SUB_ID    |Claim_Or_Rejected|claim_type      |claim_amount|claim_date|
+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
|0       |187158    |Galactosemia  |SUBID1000 |N                |claims of value |79874       |1949-03-14|
|1       |112766    |Bladder cancer|SUBID10001|NaN              |claims of policy|151142      |1970-03-16|
|2       |199252    |Kidney cancer |SUBID10002|NaN              |claims of value |59924       |2008-02-03|
|3       |133424    |Suicide       |SUBID10003|NaN              |claims of fact  |143120      |1995-02-08|
|4       |172579    |Food allergy  |SUBID10004|Y                |claims of value |168634      |1967-05-23|
+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
only showing top 5 rows



In [ ]:
data.count()

70

In [ ]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+--------+----------+------------+------+-----------------+----------+------------+----------+
|claim_id|patient_id|disease_name|SUB_ID|Claim_Or_Rejected|claim_type|claim_amount|claim_date|
+--------+----------+------------+------+-----------------+----------+------------+----------+
|       0|         0|           0|     0|               30|         0|           0|         0|
+--------+----------+------------+------+-----------------+----------+------------+----------+



In [ ]:
data.groupby(data.columns).count().where("count>1").show()

+--------+----------+------------+------+-----------------+----------+------------+----------+-----+
|claim_id|patient_id|disease_name|SUB_ID|Claim_Or_Rejected|claim_type|claim_amount|claim_date|count|
+--------+----------+------------+------+-----------------+----------+------------+----------+-----+
+--------+----------+------------+------+-----------------+----------+------------+----------+-----+



In [ ]:
data = data.dropDuplicates()

In [ ]:
data = data.replace({'NaN':None})

In [ ]:
data.show(5,False)

+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
|claim_id|patient_id|disease_name  |SUB_ID    |Claim_Or_Rejected|claim_type      |claim_amount|claim_date|
+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
|3       |133424    |Suicide       |SUBID10003|NULL             |claims of fact  |143120      |1995-02-08|
|2       |199252    |Kidney cancer |SUBID10002|NULL             |claims of value |59924       |2008-02-03|
|4       |172579    |Food allergy  |SUBID10004|Y                |claims of value |168634      |1967-05-23|
|1       |112766    |Bladder cancer|SUBID10001|NULL             |claims of policy|151142      |1970-03-16|
|0       |187158    |Galactosemia  |SUBID1000 |N                |claims of value |79874       |1949-03-14|
+--------+----------+--------------+----------+-----------------+----------------+------------+----------+
only showing top 5 rows



In [ ]:
data = data.fillna({"Claim_Or_Rejected":"NA"})

In [ ]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+--------+----------+------------+------+-----------------+----------+------------+----------+
|claim_id|patient_id|disease_name|SUB_ID|Claim_Or_Rejected|claim_type|claim_amount|claim_date|
+--------+----------+------------+------+-----------------+----------+------------+----------+
|       0|         0|           0|     0|                0|         0|           0|         0|
+--------+----------+------------+------+-----------------+----------+------------+----------+



In [ ]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/claimdatastaging.csv"%output_container_path)

True